# ESA risk list counterpart in Fink

In this notebook, we look for alerts in Fink that would correspond to identified risky asteroids from the ESA risk list (https://neo.ssa.esa.int/risk-list)

In [1]:
import requests
import pandas as pd
import io

def cross_match(oid: str, columns='i:objectId') -> int:
    """ Search alerts in Fink database corresponding to SSO designation `oid`
    
    Parameters
    ----------
    oid: str
        Name or designation for the object
    columns: str, optional
        Comma-separated column names to transfer.
        
    Returns
    ----------
    out: int
        Number of alerts matching `oid` in Fink database
    """
    r = requests.post(
        'https://fink-portal.org/api/v1/sso',
        json={
            'n_or_d': oid,
            'columns': columns,
            'output-format': 'json'
        }
    )
    
    return len(pd.read_json(io.BytesIO(r.content)))

## Loading ESA data

We just downloaded the list from https://neo.ssa.esa.int/risk-list

In [2]:
risklist = pd.read_csv('riskList.csv')
risklist

,No.,Object designator,Diameter\nin m,Impact date/time in UTC,IP max,PS max,TS,Years,IP cum,PS cum,Vel. in km/s,In list since in d
0,1,2021QM1,50*,2052-04-02 01:36,1/3115,-2.69,0,2052-2092,1/3115,-2.69,23.72,289
1,2,2010RF12,8*,2095-09-05 23:49,1/14,-3.07,0,2095-2119,1/14,-3.07,12.29,4300
2,3,1979XB,700*,2056-12-12 21:39,1/3.5E6,-3.22,0,2056-2117,1/1.11E6,-2.89,27.54,4992
3,4,2000SG344,40*,2071-09-16 00:57,1/1146,-3.37,0,2069-2119,1/371,-2.95,11.27,4992
4,5,2008JL3,30*,2027-05-01 09:06,1/6711,-3.66,0,2027-2119,1/6097,-3.65,14.01,4992
...,...,...,...,...,...,...,...,...,...,...,...,...
1371,1372,2010XC,6*,2061-03-14 13:03,1/3.75E10,-12.37,0,2061-2074,1/2.97E10,-12.29,17.13,4213
1372,1373,2016VR4,6*,2107-11-10 10:41,1/1.58E10,-12.39,0,2107,1/1.58E10,-12.39,15.43,2041
1373,1374,2019AS5,1.2*,2085-01-07 18:20,1/8.4E8,-12.65,0,2085-2111,1/3.57E8,-12.32,15.12,1253
1374,1375,2022BA7,2.4*,2117-01-27 05:41,1/7.81E9,-13.06,0,2117,1/7.81E9,-13.06,14.51,132


## Crossmatch

We simply loop over the `Object Designator` of the risk list, and see if there are corresponding alerts in Fink:

In [5]:
# takes a few minutes
oids = []
for oid in risklist['Object designator'].values:
    nobj = cross_match(oid)
    if nobj > 0:
        print(oid, '--> {} alert(s) found in Fink'.format(nobj))
        print('https://fink-portal.org/?query_type=SSO&n_or_d={}'.format(oid))
        print('https://fink-portal.org/api/v1/sso?n_or_d={}'.format(oid))
        oids.append(oid)

2020BJ14 --> 1 alert(s) found in Fink
https://fink-portal.org/?query_type=SSO&n_or_d=2020BJ14
https://fink-portal.org/api/v1/sso?n_or_d=2020BJ14


We found only one alert :-(

In [4]:
risklist[risklist['Object designator'].apply(lambda x: x in oids)]

,No.,Object designator,Diameter\nin m,Impact date/time in UTC,IP max,PS max,TS,Years,IP cum,PS cum,Vel. in km/s,In list since in d
987,988,2020BJ14,50*,2082-01-02 06:10,1/2.42E8,-8.17,0,2082,1/2.42E8,-8.17,15.4,662
